In [1]:
import os
import numpy as np
import logging

from asgardpy.analysis import AsgardpyAnalysis
from asgardpy.config import AsgardpyConfig
from asgardpy.stats import (
    check_model_preference_aic,
    fetch_all_analysis_fit_info,
    get_model_config_files,
    tabulate_best_fit_stats,
    copy_target_config,
)

In [ ]:
# os.environ['GAMMAPY_DATA']

In [2]:
log = logging.getLogger("Check Preferred Spectral Model")

In [3]:
config_file = "../src/asgardpy/tests/config_test_gadf.yaml"

# Initiate the main "base" config

In [4]:
main_config = AsgardpyConfig.read(config_file)
target_source_name = main_config.target.source_name

In [5]:
spec_model_temp_files = get_model_config_files(["lp", "bpl", "ecpl", "ecpl2", "pl", "eclp", "sbpl", "secpl"])

In [6]:
main_config.dataset3d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}hess-dl3-dr1/"
main_config.dataset3d.instruments[0].dataset_info.background.exclusion.exclusion_file = (
    f"{os.environ['GAMMAPY_DATA']}joint-crab/exclusion/exclusion_mask_crab.fits.gz"
)
main_config.dataset1d.instruments[0].input_dl3[0].input_dir = f"{os.environ['GAMMAPY_DATA']}magic/rad_max/data/"

In [7]:
def fetch_all_analysis_objects(main_config, spec_model_temp_files):
    """For a list of spectral models, initiate AsgardpyAnalysis objects."""
    main_analysis_list = {}
    spec_models_list = []

    for temp in spec_model_temp_files:
        temp_model = AsgardpyAnalysis(main_config)
        temp_model.config.target.models_file = temp

        temp_model_2 = AsgardpyAnalysis(temp_model.config)

        copy_target_config(temp_model, temp_model_2)

        spec_tag = temp.name.split(".")[0].split("_")[-1]
        spec_models_list.append(spec_tag)
        main_analysis_list[spec_tag] = {}

        main_analysis_list[spec_tag]["Analysis"] = temp_model_2

    spec_models_list = np.array(spec_models_list)

    return main_analysis_list, spec_models_list

In [8]:
%%time
main_analysis_list, spec_models_list = fetch_all_analysis_objects(
    main_config, spec_model_temp_files
)

Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filemode': 'w', 'format': '', 'datefmt': ''}
Setting logging config: {'level': 'INFO', 'filename': '', 'filem

CPU times: user 152 ms, sys: 0 ns, total: 152 ms
Wall time: 149 ms


# Run Analysis steps

In [9]:
%%time
PL_idx = 0

for i, tag in enumerate(spec_models_list):
    log.info("Spectral model being tested: %s", tag)

    main_analysis_list[tag]["Analysis"].run()

    if tag == "pl":
        PL_idx = i


Spectral model being tested: lp
Perform DL3 to DL4 process!
1 number of 3D Datasets given
No distinct keys used for the 3D dataset
Observation ID list selected: 23523 23526 23559
Observations selected: 3 out of 3.
Using 3 processes to compute Data reduction
Computing dataset for observation 23523
Running MapDatasetMaker
Computing dataset for observation 23526
Running MapDatasetMaker
Computing dataset for observation 23559
Running MapDatasetMaker
Running SafeMaskMaker
Running FoVBackgroundMaker
Running SafeMaskMaker
Running FoVBackgroundMaker
Running SafeMaskMaker
Running FoVBackgroundMaker
Analysis Step datasets-3d completed
1 number of 1D Datasets given
Observation ID list selected: 5029747 5029748
Observations selected: 2 out of 2.
Using 2 processes to compute Data reduction
Computing dataset for observation 5029747
Computing dataset for observation 5029748
Running SpectrumDatasetMaker
Running SpectrumDatasetMaker
'THETA' axis is stored as a scalar -- converting to 1D array.
'THETA' 

ValueError: 'Crab Nebula' is not in list

# Get all the Fit results

In [ ]:
%%time
fit_success_list, stat_list, dof_list, pref_over_pl_chi2_list = fetch_all_analysis_fit_info(
    main_analysis_list, spec_models_list
)

In [10]:
# If any spectral model has at least 5 sigmas preference over PL
best_sp_idx_lrt = np.nonzero(pref_over_pl_chi2_list == np.nanmax(pref_over_pl_chi2_list))[0]
for idx in best_sp_idx_lrt:
    if pref_over_pl_chi2_list[idx] > 5:
        sp_idx_lrt = idx
        log.info("Best preferred spectral model over PL is %s", spec_models_list[idx])
    else:
        sp_idx_lrt = PL_idx
        log.info("No other model preferred over PL")


NameError: name 'pref_over_pl_chi2_list' is not defined

In [ ]:
list_rel_p = check_model_preference_aic(stat_list, dof_list)

In [ ]:
best_sp_idx_aic = np.nonzero(list_rel_p == np.nanmax(list_rel_p))[0]

for idx in best_sp_idx_aic:
    if list_rel_p[idx] > 0.95:
        sp_idx_aic = idx
        log.info("Best preferred spectral model is %s", spec_models_list[fit_success_list][idx])
    else:
        sp_idx_aic = PL_idx
        log.info("No other model preferred, hence PL is selected")

# Write the results into a Table

In [ ]:
stats_table = tabulate_best_fit_stats(spec_models_list, fit_success_list, main_analysis_list, list_rel_p)

stats_table.meta["Target source name"] = target_source_name

In [ ]:
stats_table